In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import import_ipynb
from rotation import rotation_matrix, unit_vector, angle_between, x_rotation, y_rotation, z_rotation

# for machine learning
from sklearn import model_selection, preprocessing, feature_selection, ensemble, linear_model, metrics, decomposition

from csv import reader
import pickle

importing Jupyter notebook from rotation.ipynb


In [2]:
NUM_FEATURES = 3
NUM_JOINTS = 20
NUM_FRAMES = 16
#FILE_NAME = '../train.csv'
FILE_NAME = '../test.csv'

In [3]:
dtf = pd.read_csv(FILE_NAME, header = None)
dtf = dtf.set_index(dtf.columns[0])

dtf = dtf.sample(frac = 1)
X_train = dtf.iloc[:,:-1]
Y_train = dtf.iloc[:,-1:]
X_train

,1,2,3,4,5,6,7,8,9,10,...,950,951,952,953,954,955,956,957,958,959
0,,,,,,,,,,,,,,,,,,,,,
2019,-13.339000,-7.50750,-13.92800,-3.43030,-7.910200,-8.429000,5.175900,-6.635000,-2.810900,7.872500,...,5.126900,-14.2690,-4.8379,-10.12900,25.6510,-4.38380,-4.58040,4.63250,-3.44720,-1.03830
363,-1.177200,6.66570,1.13250,0.21072,3.174300,0.391660,0.028295,1.366400,-0.320930,-0.008935,...,-0.014148,33.9690,-5.8682,-0.89278,-3.3723,-3.50930,-4.32170,-3.72010,-14.73200,-6.28670
228,-0.092305,0.33093,-0.25503,-0.14222,0.086151,0.061536,-0.462210,-0.093672,0.380160,0.129230,...,-27.384000,16.1680,7.1321,-5.26620,5.7926,-1.40710,-0.76442,3.77150,-0.84237,1.42560
55,-2.900400,2.06780,-4.24870,-3.57360,0.075636,-5.432700,-2.947900,-0.353180,-5.455800,-1.180200,...,15.253000,2.2828,-2.1291,8.80500,-6.9716,2.29410,1.53210,3.86810,-3.89310,-0.48444
2591,0.070044,0.61271,1.82400,0.35447,0.387720,1.712200,0.549740,-0.007075,1.374700,1.164600,...,4.099300,2.9801,-1.5686,-2.15160,3.3572,-0.96222,0.68275,0.63606,-3.33030,1.08250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,-15.402000,-9.19770,-14.71400,-5.64560,-0.228170,-1.151800,-3.733100,0.852770,4.145900,-6.674500,...,38.352000,-63.5030,-16.2250,56.42800,-207.0800,-6.79980,64.39600,-190.95000,0.18581,63.42900
190,-0.043955,5.68480,-1.58490,0.74150,2.162700,-1.867100,1.981200,-0.424260,-2.479300,-2.426400,...,0.838970,-6.7429,2.3793,-0.20512,-3.8024,1.23970,-2.35130,-1.18040,-3.24440,-3.52660
2131,11.028000,1.99820,-1.55290,12.88200,-0.106860,-0.478140,13.943000,-2.989400,0.017125,18.573000,...,6.755600,-151.5200,-8.4709,12.53600,-220.0800,23.41700,2.74900,-154.40000,13.41900,2.12420


In [4]:
def read_xyz(row):
    skeleton_data, label = read_skeleton(row)
    
    data = np.zeros((NUM_FRAMES, NUM_JOINTS, NUM_FEATURES))
    for m, i in enumerate(skeleton_data['frame_info']):
        for n, j in enumerate(i['joint_info']):
            feature_info = j['feature_info']
            data[m, n, :] = [feature_info['x'], feature_info['y'], feature_info['z']]

    data = data.transpose(2, 0, 1)
    return data, label

In [5]:
def read_skeleton(row):
    data = row[1:]
    #label = row[-1]
    label = -1
    skeleton_data = {}
    skeleton_data['index'] = row[0]
    skeleton_data['num_frame'] = NUM_FRAMES
    skeleton_data['frame_info'] = []
    
    for frame in range(NUM_FRAMES):
        offset = NUM_JOINTS * NUM_FEATURES
        data_in_frame = row[1+frame*offset:1+(frame+1)*offset]
        frame_info = {}
        frame_info['num_joints'] = NUM_JOINTS
        frame_info['joint_info'] = []
        
        for feature in range(NUM_JOINTS):
            offset = NUM_FEATURES
            data_in_joint = data_in_frame[feature*offset:(feature+1)*offset]
            joint_info = {}
            joint_info['num_features'] = NUM_FEATURES
            joint_info['feature_info'] = {
                k: float(v)
                for k, v in zip(['x', 'y', 'z'], data_in_joint)
            }
            frame_info['joint_info'].append(joint_info)
                    
        skeleton_data['frame_info'].append(frame_info)
    return skeleton_data, label

In [6]:
def normalisation(data):
    N, C, T, V, M = data.shape
    s = np.transpose(data, [0, 4, 2, 3, 1])
    zaxis=[0, 1]
    xaxis=[8, 4]
    
    print('sub the center joint #1 (spine joint in ntu and neck joint in kinetics)')
    for i_s, skeleton in enumerate(tqdm(s)):
        if skeleton.sum() == 0:
            continue
        main_body_center = skeleton[0][:, 1:2, :].copy()
        for i_p, person in enumerate(skeleton):
            if person.sum() == 0:
                continue
            mask = (person.sum(-1) != 0).reshape(T, V, 1)
            s[i_s, i_p] = (s[i_s, i_p] - main_body_center) * mask

    print('parallel the bone between hip(jpt 0) and spine(jpt 1) of the first person to the z axis')
    for i_s, skeleton in enumerate(tqdm(s)):
        if skeleton.sum() == 0:
            continue
        joint_bottom = skeleton[0, 0, zaxis[0]]
        joint_top = skeleton[0, 0, zaxis[1]]
        axis = np.cross(joint_top - joint_bottom, [0, 0, 1])
        angle = angle_between(joint_top - joint_bottom, [0, 0, 1])
        matrix_z = rotation_matrix(axis, angle)
        for i_p, person in enumerate(skeleton):
            if person.sum() == 0:
                continue
            for i_f, frame in enumerate(person):
                if frame.sum() == 0:
                    continue
                for i_j, joint in enumerate(frame):
                    s[i_s, i_p, i_f, i_j] = np.dot(matrix_z, joint)

    print('parallel the bone between right shoulder(jpt 8) and left shoulder(jpt 4) of the first person to the x axis')
    for i_s, skeleton in enumerate(tqdm(s)):
        if skeleton.sum() == 0:
            continue
        joint_rshoulder = skeleton[0, 0, xaxis[0]]
        joint_lshoulder = skeleton[0, 0, xaxis[1]]
        axis = np.cross(joint_rshoulder - joint_lshoulder, [1, 0, 0])
        angle = angle_between(joint_rshoulder - joint_lshoulder, [1, 0, 0])
        matrix_x = rotation_matrix(axis, angle)
        for i_p, person in enumerate(skeleton):
            if person.sum() == 0:
                continue
            for i_f, frame in enumerate(person):
                if frame.sum() == 0:
                    continue
                for i_j, joint in enumerate(frame):
                    s[i_s, i_p, i_f, i_j] = np.dot(matrix_x, joint)

    data = np.transpose(s, [0, 4, 2, 3, 1])
    return data
    

In [7]:
fp = np.zeros((len(dtf), NUM_FEATURES, NUM_FRAMES, NUM_JOINTS, 1), dtype=np.float32)
    #construct a matrix, with num of data, num of features for each joint, num of frames, num of joints, num of people(always 1 in our case)
with open(FILE_NAME, 'r') as f:
        csv_reader = reader(f)
        labels = []
        for i, row in enumerate(csv_reader):
            data, label = read_xyz(row)
            labels.append(label)
            fp[i, :, :, :, 0] = data
fp = normalisation(fp)
#np.save('train_data.npy', fp)
np.save('test_data.npy', fp)

# with open('label.pkl', 'wb') as f:
#     pickle.dump(labels, f)

sub the center joint #1 (spine joint in ntu and neck joint in kinetics)


100%|███████████████████████████████████████████████████████████████████████████| 2959/2959 [00:00<00:00, 33720.17it/s]


parallel the bone between hip(jpt 0) and spine(jpt 1) of the first person to the z axis


100%|████████████████████████████████████████████████████████████████████████████| 2959/2959 [00:02<00:00, 1177.35it/s]


parallel the bone between right shoulder(jpt 8) and left shoulder(jpt 4) of the first person to the x axis


100%|████████████████████████████████████████████████████████████████████████████| 2959/2959 [00:02<00:00, 1223.45it/s]


In [8]:
unpickled_df = pd.read_pickle("label_test.pkl")
unpickled_df

FileNotFoundError: [Errno 2] No such file or directory: 'label_test.pkl'